<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [2]:
import json
import logging
import random
import time
from spire.doc import *
from spire.doc.common import *
import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *
import datetime
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [3]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [4]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [5]:
vocab_dict

'しゅだん,いいん,かがやく,ふそく,かんする,もじ,のき,じゅうたい,びん,ぎむ,よん,ひひょう,こくご,けいじ,けしょう,とどく,こううん,たまる,そしき,ちょうさ,じさつ,あいず,じじつ,りこん,てんこう,しばい,かもく,にんき,いずみ,きのう,みりょく,せんこう,くべつ,おい,さんこう,かける,むす,つれ,し,いま,ねんれい,きず,こい,くだる,かぐ,きゅうそく,かげ,いどう,あい,ふか,はた,か,ほしょう,しょさい,どうし,そろえる,もうふ,このみ,いわう,ぜいたく,がん,まかせる,とし,ちしき,しょくぶつ,いわ,かんそう,きほん,す,すう,ちゅうもく,かつやく,かかえる,けってん,つち,しゅう,うしなう,すいじゅん,しんぱん,つける,せいげん,おくれ,へる,むく,あかり,てつや,ゆうこう,ここ,ちがい,ちょくせつ,いじわる,こうふく,め,たいら,ぎん,しゅうきょう,とうひょう,ふたご,したがう,つみ,しぜん,ちょうせん,ほう,まご,おとる,たんなる,やとう,りそう,はくぶつかん,あらし,みやげ,いっぽう,わる,ごがく,いんさつ,ほほえむ,ころぶ,たいしょう,おそらく,くむ,おまえ,たすける,じしん,じこく,れんそう,しょうひん,じつりょく,ほぼ,まぜる,たんじゅん,しんゆう,あける,へいわ,ふんいき,しんがく,かかく,つよめる,もうしわけ,とびだす,のうか,む,はじまり,もと,けんちく,ぜい,ぐんたい,おもいで,やね,する,としつき,じゅよう,ぬく,ぼう,しょう,そだつ,さいこう,へんこう,ようそ,さっきょく,かんせい,てきよう,らく,きんせん,しゅうかく,こうしゃ,じんしゅ,すれちがう,く,ぎょうぎ,めいし,しんよう,かくだい,わかれ,ねんかん,きんし,いためる,とじる,あらわす,ちゅう,かく,うつる,ぞう,なんで,てき,やく,くんれん,じゅうし,なべ,まざる,たいよう,さんそ,えいせい,わずか,こい,いとこ,じょし,さべつ,とく,にんげん,そで,おんど,うまれ,なみだ,すみ,ならす,おう,くわえる,くつう,なし,たに,いがい,おろす,れっしゃ,たがい,しょくもつ,ことわる,おそれる,うつる,ほね,あん,げんきん,おう,とうよう,もよう,いだい,そくど,たんじょう,かくじつ,ちる,ゆか,ようてん,うわさ,しゅうり,きり,たいする,けんかい,まく,かたづ

#### load Models

In [6]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_kanji): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- 問題1 のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading): 1 questions
- 問題2 このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_kanji): 1 questions
- 問題3（　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。 (word_meaning): 1 questions
- 問題4 に意味が最も近いものを、１・２・３・４から一つえらびなさい。(synonym_substitution) 1 questions
- 問題5 つぎのことばの使い方として最もよいものを、２・３・４から一つえらびなさい。 (word_usage): 1 questions
Section 2: Reading Comprehension
- Information retrieval (info_retrieval): 0 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 0 questions

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The topic word in outline must be presented in Japanese kana only. The complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of moderate and very difficult topics to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First, evenly pick Japanese words used in different part of sentence from 'Vocabulary' for questions in Vocabulary and Grammar, ensuring 40 percent of very difficult words for topics. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [7]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [8]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [9]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-06-02 21:10:20,904 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [10]:
initial_outline

Outline(page_title='日本語能力試験N3 模擬試験問題用紙', sections=[Section(section_title='第1部\u3000語彙・文法', subsections=[Subsection(subsection_title='(kanji_reading)', description='問題1：ことばの漢字の読み方を選ぶ（1問）', question_topics=[QuestionTopic(topic='かんする')]), Subsection(subsection_title='(write_kanji)', description='問題2：ことばを漢字で書く（1問）', question_topics=[QuestionTopic(topic='しゅだん')]), Subsection(subsection_title='(word_meaning)', description='問題3：（\u3000）に入れる最もよいことばを選ぶ（1問）', question_topics=[QuestionTopic(topic='ふそく')]), Subsection(subsection_title='(synonym_substitution)', description='問題4：意味が最も近いことばを選ぶ（5問）', question_topics=[QuestionTopic(topic='かんする'), QuestionTopic(topic='しゅだん'), QuestionTopic(topic='ふそく'), QuestionTopic(topic='じゅうたい'), QuestionTopic(topic='かんする')]), Subsection(subsection_title='(word_usage)', description='問題5：ことばの使い方として最もよいものを選ぶ（5問）', question_topics=[QuestionTopic(topic='じゅうたい'), QuestionTopic(topic='ふそく'), QuestionTopic(topic='しゅだん'), QuestionTopic(topic='かんする'), QuestionTopic(topic='しゅだ

In [11]:
from IPython.display import display, Markdown, HTML
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験問題用紙

## 第1部　語彙・文法

### (kanji_reading)

問題1：ことばの漢字の読み方を選ぶ（1問）

- **かんする**

### (write_kanji)

問題2：ことばを漢字で書く（1問）

- **しゅだん**

### (word_meaning)

問題3：（　）に入れる最もよいことばを選ぶ（1問）

- **ふそく**

### (synonym_substitution)

問題4：意味が最も近いことばを選ぶ（5問）

- **かんする**
- **しゅだん**
- **ふそく**
- **じゅうたい**
- **かんする**

### (word_usage)

問題5：ことばの使い方として最もよいものを選ぶ（5問）

- **じゅうたい**
- **ふそく**
- **しゅだん**
- **かんする**
- **しゅだん**

In [12]:
import re
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = re.findall(r'\((.*?)\)', subsection['subsection_title'])[0]

        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing (kanji_reading):   0%|          | 0/1 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-06-02 21:10:27,866 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-02 21:10:27,867 - INFO - Generated message: <a>この問題は環境に<u>関する</u>ものです。</a>
<ul>
    <li>かんせる</li>
    <li>かんする</li>
    <li>かんしゅう</li>
    <li>かんしゃ</li>
</ul>
正解: 2


---REVISOR---


2025-06-02 21:10:30,417 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-02 21:10:30,418 - INFO - Refelect message: Content Accuracy:  
The vocabulary item "関する" is appropriate for the JLPT N3 level. The sample sentence "この問題は環境に関するものです。" is clear, correctly formed, and relevant to common academic and everyday contexts. The distractor options (かんせる, かんしゅう, かんしゃ) are plausible and at a similar level, which is good for testing recognition.

Question and Answer Quality:  
The question is unambiguous and tests the student's understanding of the correct word in context. The answer choices are distinct, and only one is correct, which aligns with best practices for multiple-choice questions.

Cultural and Academic Relevance:  
The sentence is culturally neutral and academically appropriate, referencing "environmental issues," which is a common topic in Japa

...

---REVISOR---


2025-06-02 21:12:36,094 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-02 21:12:36,095 - INFO - Refelect message: Content Accuracy:  
The vocabulary item "手段" (means, method, way) is appropriate for JLPT N3 level. The example sentences (1-3) accurately reflect the correct usage of "手段" in natural, everyday contexts, as would be expected at this level. Sentence 4 is incorrect, as "手段" does not mean "manner" in the sense of kindness; the word "方法" or "やり方" would be more appropriate if referring to "way" in terms of kindness. This makes sentence 4 the correct answer for identifying the incorrect usage, which is appropriate for this type of question.

Question and Answer Quality:  
The wording of each sentence is clear, concise, and unambiguous. The sentences effectively test the student's understanding of the meaning and usage of "手段." The distractors

--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-06-02 21:12:37,118 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-02 21:12:37,119 - INFO - Formatted message: {'html_question': '<a>手段</a>', 'correct_answer': 4, 'choices': ['目的のためには<u>手段</u>を選ばない人もいます。', 'この問題を解決する<u>手段</u>を考えましょう。', '駅まで行く<u>手段</u>がなくて、困っています。', '彼の<u>手段</u>はとても親切でした。']}

Processing 第1部　語彙・文法: 100%|██████████| 5/5 [02:16<00:00, 27.23s/it]

Total execution time: 136.17 seconds


In [13]:
# yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# # Display as formatted YAML
# display(Markdown(yaml_str))

In [14]:
output_data

{'sections': [{'section_title': '第1部\u3000語彙・文法',
   'subsections': [{'subsection_title': '(kanji_reading)',
     'description': '問題1：ことばの漢字の読み方を選ぶ（1問）',
     'question_topics': [{'topic': 'かんする',
       'result': {'html_question': '<a>この問題は環境に<u>関する</u>ものです。</a>',
        'correct_answer': 2,
        'choices': ['かんせる', 'かんする', 'かんしゅう', 'かんしゃ']}}]},
    {'subsection_title': '(write_kanji)',
     'description': '問題2：ことばを漢字で書く（1問）',
     'question_topics': [{'topic': 'しゅだん',
       'result': {'html_question': '<a>この問題を解決する<u>しゅだん</u>を考えましょう。</a>',
        'correct_answer': 1,
        'choices': ['手段', '主段', '主断', '手断']}}]},
    {'subsection_title': '(word_meaning)',
     'description': '問題3：（\u3000）に入れる最もよいことばを選ぶ（1問）',
     'question_topics': [{'topic': 'ふそく',
       'result': {'html_question': '<a>最近、この町では水が（\u3000）している。</a>',
        'correct_answer': 2,
        'choices': ['減少', '不足', '配達', '増加']}}]},
    {'subsection_title': '(synonym_substitution)',
     'description': '問題4：意味が最も近い

In [15]:
def render_to_html(sections):
    html = '<html><body style="text-align: left;">\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for idx, qtopic in enumerate(subsection.get("question_topics", []), 1):
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += '<div>\n'
                        # html += f'<div style="display: flex; align-items: center;"><div>{idx}.  </div>{result["html_question"]}</div>\n'
                        html += f'<p><strong>{idx}. </strong>{result["html_question"]}</p>'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li>{idx}. <b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{idx}. {choice}</li>\n"
                            html += '</ul>\n'
                        html += '</div>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [16]:
html_output = render_to_html(output_data['sections'])
display(HTML(html_output))

In [17]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = "output"
html_filename = f"JLPT_{timestamp}.html"
html_filepath = os.path.join(output_folder,html_filename)
with open(html_filepath, "w", encoding="utf-8") as file:
    file.write(html_output)

In [18]:
# Create a Document object
document = Document()
# Add a section to the document
section = document.AddSection()
# Set the page margins to 72 points (72 points = 1 inch)
section.PageSetup.Margins.All = 72

# Add a paragraph to the section
paragraph = section.AddParagraph()

# Add the HTML string to the paragraph
paragraph.AppendHTML(html_output)

doc_filename = f"JLPT_{timestamp}.docx"
doc_filepath = os.path.join(output_folder,doc_filename)
# Save the result document to a DOCX file
document.SaveToFile(doc_filepath, FileFormat.Docx2016)
# Or save the result document to a DOC file
# document.SaveToFile("HtmlStringToDoc.doc", FileFormat.Doc)

document.Close()

In [19]:
import paramiko

# Load the private key from the environment variable
# Parse the private key
key_file_path = 'temp_id_ed25519'
pkey = paramiko.Ed25519Key.from_private_key_file(key_file_path)

# SSH and SFTP parameters
hostname = "58.246.203.10"
username = "hwu"
port = 22

remote_html_file_path = f"/var/www/html/{html_filename}"
remote_doc_file_path = f"/var/www/html/{doc_filename}"

# Establish SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname=hostname, port=port, username=username, pkey=pkey)

# Transfer the file
sftp = ssh.open_sftp()
sftp.put(html_filepath, remote_html_file_path)
print(f"File transferred to {hostname}:{remote_html_file_path}")
sftp.put(doc_filepath, remote_doc_file_path)
print(f"File transferred to {hostname}:{remote_doc_file_path}")

# Clean up
sftp.close()
ssh.close()

2025-06-02 21:12:37,637 - INFO - Connected (version 2.0, client OpenSSH_9.6p1)
2025-06-02 21:12:37,922 - INFO - Authentication (publickey) successful!
2025-06-02 21:12:38,215 - INFO - [chan 0] Opened sftp connection (server version 3)
2025-06-02 21:12:38,629 - INFO - [chan 0] sftp session closed.


File transferred to 58.246.203.10:/var/www/html/JLPT_20250602_211237.html
File transferred to 58.246.203.10:/var/www/html/JLPT_20250602_211237.docx
